In [ ]:
%tensorflow_version 1.14.0
from google.colab import drive

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/sunil/concepts_loss_origin/concept_learning/

/content/gdrive/MyDrive/sunil/concepts_loss_origin/concept_learning


In [ ]:
# !git diff --name-only

In [ ]:
# !git checkout .
# !git pull

In [ ]:
import tensorflow as tf
from clearn.experiments.experiment import initialize_model_train_and_get_features, MODEL_TYPE_VAE_SEMI_SUPERVISED_MNIST
from clearn.config import ExperimentConfig
from clearn.utils.data_loader import load_images



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# !rm -rf /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_unsup_seg_fmnist_5/Exp_128_256_256_512_16_5


In [ ]:
from clearn.experiments.experiment import Experiment, initialize_model_train_and_get_features,  MODEL_TYPE_VAE_SEMI_SUPERVISED_MNIST


In [ ]:
create_split = True
experiment_name = "exp_concepts_1"
num_epochs = 50
run_id = 1
z_dim = 16
train_val_data_iterator = None
root_path="/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp"


In [ ]:
num_units = [512, 256, 256, 128, 64]
stride_5_layer = [2, 1, 2, 1, 2,  1]


In [ ]:
layer_num=len(num_units) - 2
label_list = list(range(10,28))
print(label_list)
print(layer_num)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
3


In [ ]:
exp_config = ExperimentConfig(root_path=root_path,
                              num_decoder_layer=4,
                              num_cluster_config=None,
                              supervise_weight=0,
                              name=experiment_name,
                              z_dim=z_dim,
                              batch_size=512,
                              beta=5,
                              reconstruction_weight=1,
                              num_units=num_units,
                              save_reconstructed_images=True,
                              split_name="split_70_30",
                              dataset_name="mnist_concepts",
                              eval_interval_in_epochs=4,
                              model_save_interval=10,
                              num_val_samples=-1,
                              run_evaluation_during_training=True,
                              return_latent_vector=False,
                              write_predictions=True,
                                                                                           fully_convolutional = True,
                                                                                           num_concepts=28,
                              strides=stride_5_layer,
                                                                                           activation_output_layer = "SIGMOID",
                                                                                           num_dense_layers=0,
                                                                                           uncorrelated_features=False,
                              concept_id=20,
                              concept_dict = { 6: {"unique_concepts":[-1]},
                                              layer_num:{"unique_concepts":label_list
                                                },
                                              },

                                                                                            )
exp_config.check_and_create_directories(run_id)


/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/datasets/ split_70_30 manually_generated_concepts.json
Reading concepts map from /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/datasets//mnist_concepts/split_70_30/manually_generated_concepts.json
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


True

In [ ]:
exp_config.ANALYSIS_PATH

'/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/exp_concepts_1/Exp_64_128_256_256_512_16_1/analysis/'

In [ ]:
from clearn.analysis.cluster_utils import get_manual_annotation_col_name
import pandas as pd

In [ ]:
exp_config.dao.num_classes

28

In [ ]:
exp_config.dao.num_concepts_label_generated

18

In [ ]:
from clearn.utils.utils import show_all_variables, get_padding_info

def get_num_concepts_per_image(exp_config, dao):

    _,  _, image_sizes = get_padding_info(exp_config, dao.image_shape)
    latent_image_dim = image_sizes[len(exp_config.num_units)]
    concepts_stride = 1

    if latent_image_dim[0] % concepts_stride == 0:
        h = latent_image_dim[0] //concepts_stride
    else:
        h = (latent_image_dim[0] // concepts_stride) + 1
    if latent_image_dim[1] % concepts_stride == 0:
        w = latent_image_dim[1] // concepts_stride
    else:
        w = (latent_image_dim[1] // concepts_stride) + 1
    return h, w


In [ ]:
from clearn.dao.idao import IDao
from clearn.utils.data_loader import TrainValDataIterator, DataIterator
import os
def get_train_val_iterator(create_split: bool,
                           dao: IDao,
                           exp_config: ExperimentConfig,
                           num_epochs_completed: int,
                           split_name: str):
    split_filename = exp_config.DATASET_PATH + split_name + ".json"
    manual_annotation_file_name = f"manual_annotation.csv"

    manual_annotation_file = os.path.join(exp_config.ANALYSIS_PATH,
                                          manual_annotation_file_name
                                          )
    if exp_config.fully_convolutional:
        num_concepts_per_row, num_concepts_per_col = get_num_concepts_per_image(exp_config, dao)
    else:
        num_concepts_per_row, num_concepts_per_col = 1, 1
    # if os.path.isfile(split_filename):
    #     print("File exists", split_filename)
    #     if manual_annotation_file is not None:
    #         train_val_data_iterator = TrainValDataIterator.from_existing_split(dao=dao,
    #                                                                            split_name=exp_config.split_name,
    #                                                                            split_location=exp_config.DATASET_PATH,
    #                                                                            batch_size=exp_config.BATCH_SIZE,
    #                                                                            manual_labels_config=exp_config.manual_labels_config,
    #                                                                            manual_annotation_file=manual_annotation_file,
    #                                                                            budget=exp_config.budget,
    #                                                                            num_concepts_per_image_row=num_concepts_per_row,
    #                                                                            num_concepts_per_image_col=num_concepts_per_col
    #                                                                            )
    if True:
        train_val_data_iterator = TrainValDataIterator(dataset_path=exp_config.DATASET_ROOT_PATH,
                                                       dao=dao,
                                                       shuffle=True,
                                                       stratified=True,
                                                       validation_samples=exp_config.num_val_samples,
                                                       split_names=["train", "validation"],
                                                       split_location=exp_config.DATASET_PATH,
                                                       batch_size=exp_config.BATCH_SIZE,
                                                       manual_labels_config=exp_config.manual_labels_config,
                                                       manual_annotation_file=manual_annotation_file,
                                                       seed=exp_config.seed,
                                                       budget=exp_config.budget,
                                                       num_concepts_per_image_row=num_concepts_per_row,
                                                       num_concepts_per_image_col=num_concepts_per_col
                                                       )
    else:
        raise Exception(f"File does not exists {split_filename}")
    return train_val_data_iterator


In [ ]:
exp_config.ANALYSIS_PATH

'/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/exp_concepts_1/Exp_64_128_256_256_512_16_1/analysis/'

In [ ]:
# !rm -rf /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_supervised_vae_seg_mnist/Exp_64_128_256_256_512_16_22

In [ ]:
train_val_data_iterator = get_train_val_iterator(create_split=True,
                                                dao= exp_config.dao,
                                                exp_config= exp_config,
                                                num_epochs_completed=0,
                                                split_name=exp_config.split_name
                                                 )


Image shape [28, 28, 1]
[(28, 28, 1), (14, 14, 512), (14, 14, 256), (7, 7, 256), (7, 7, 128), (4, 4, 64), 16]
[512, 256, 256, 128, 64]
<class 'numpy.ndarray'> int64
Loaded manual annotation
Number of samples with manual confidence 73414
Warning /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/exp_concepts_1/Exp_64_128_256_256_512_16_1/analysis/manual_annotation.csv path does not exist. Creating random prior with uniform distribution


In [ ]:
df = pd.read_csv(exp_config.root_path+ "/datasets/" + f"mnistconcept_{exp_config.concept_id}.csv")

In [ ]:
exp_config.root_path+ "/datasets/" + f"mnistconcept_{exp_config.concept_id}.csv"

'/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/datasets/mnistconcept_20.csv'

In [ ]:
!ls -ltr /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/datasets/mnistconcept_20.csv

-rw------- 1 root root 350403096 Jun 20 09:59 /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/datasets/mnistconcept_20.csv


In [ ]:
df.shape

(104878, 785)

In [ ]:
df[df["label"] > 9].shape

(44878, 785)

In [ ]:
# -1 means compute the loss at output layer
df["apply_loss_at_layer"] = -1
df.loc[df["label"] > 9, "apply_loss_at_layer"] = len(num_units) - 2


In [ ]:
df["label"].max()

27

In [ ]:
#df[df["apply_loss_at_layer"] == 4][["label", "apply_loss_at_layer"]].head(100)

In [ ]:
out_path = exp_config.ANALYSIS_PATH

In [ ]:
out_path

'/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/exp_concepts_1/Exp_64_128_256_256_512_16_1/analysis/'

In [ ]:
!ls /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/exp_concepts_1/Exp_64_128_256_256_512_16_1/analysis/

manual_annotation.csv


In [ ]:
import pandas as pd
import os
manual_annotation_file = os.path.join(out_path, "manual_annotation.csv")
# epoch_col_confidence, epoch_col_label = get_manual_annotation_col_name(0.0)

manual_annotation_df = pd.DataFrame(df["label"].values,
                                    columns=["manual_annotation"] )
manual_annotation_df["manual_annotation_confidence"] = 1
manual_annotation_df["apply_loss_at_layer"] = df["apply_loss_at_layer"].values

manual_annotation_df.to_csv(manual_annotation_file, index=False)


In [ ]:
manual_annotation_df.shape

(104878, 3)

In [ ]:
import numpy as np
np.max(manual_annotation_df["manual_annotation"])

NameError: ignored

In [ ]:
exp_config.dao.num_classes

28

In [ ]:
exp_config.dao.number_of_training_samples

180001

In [ ]:
!ls /content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_supervised_vae_seg_mnist

Exp_64_128_256_256_512_16_2


In [ ]:
# !rm -rf "/content/gdrive/MyDrive/concept_learning/concept_learning/concept_learning_exp/find_architecture_supervised_vae_seg_mnist/Exp_128_256_256_512_16_1"


In [ ]:

# train_images, train_labels, _ = load_images(exp_config,
#                                           train_val_data_iterator,
#                                           "train"
#                                           )
#train_labels= np.argwhere(train_labels == 1)[:,1]

In [ ]:
train_val_data_iterator.reset_counter("train")

In [ ]:
train_val_data_iterator.get_num_samples("train")

73414

In [ ]:
df.shape

(104878, 785)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,252.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


In [ ]:
import numpy as np
print(df.shape)
np.max(df.values[:, 0:784])

(104878, 785)


255.0

In [ ]:
np.max(train_val_data_iterator.train_x)

1.0

In [ ]:
#!git diff --name-only

In [ ]:
train_val_data_iterator.manual_annotation.shape

(104878, 29)

In [ ]:
df.head(100)